<a href="https://colab.research.google.com/github/bhstoller/multi-agent-customer-service/blob/main/agent_to_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Multi-Agent Customer Service System with A2A and MCP**

This notebook demonstrates agent-to-agent (A2A) communication using Google ADK with MCP (Model Context Protocol) tools.

## System Components
- **Customer Data Agent**: Interfaces with MCP server to fetch/update customer data
- **Support Agent**: Handles customer support queries and troubleshooting
- **Router**: Orchestrates communication between agents based on query type

## Environment Setup

Installing required packages and dependencies.

In [2]:
!pip install uv -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 56.0 MB/s eta 0:00:00


In [3]:
# Install required packages
%pip install --upgrade -q google-genai google-adk==1.9.0 a2a-sdk==0.3.0 python-dotenv aiohttp uvicorn requests mermaid-python nest-asyncio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.3/130.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.5/260.5 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


## Import Dependencies

Loading required libraries for A2A communication, MCP integration, and server setup.

In [4]:
import sys
from a2a.client import client as real_client_module
from a2a.client.card_resolver import A2ACardResolver

class PatchedClientModule:
    def __init__(self, real_module) -> None:
        for attr in dir(real_module):
            if not attr.startswith('_'):
                setattr(self, attr, getattr(real_module, attr))
        self.A2ACardResolver = A2ACardResolver


patched_module = PatchedClientModule(real_client_module)
sys.modules['a2a.client.client'] = patched_module  # type: ignore

In [57]:
import sys
from a2a.client import client as real_client_module
from a2a.client.card_resolver import A2ACardResolver
import asyncio
import logging
import os
import sys
import threading
import time
from typing import Any
import httpx
import nest_asyncio
import uvicorn
from a2a.client import ClientConfig, ClientFactory, create_text_message_object
from a2a.server.apps import A2AStarletteApplication
from a2a.server.request_handlers import DefaultRequestHandler
from a2a.server.tasks import InMemoryTaskStore
from a2a.types import (
    AgentCapabilities,
    AgentCard,
    AgentSkill,
    TransportProtocol,
)
from a2a.utils.constants import AGENT_CARD_WELL_KNOWN_PATH
from dotenv import load_dotenv
from google.adk.a2a.executor.a2a_agent_executor import (
    A2aAgentExecutor,
    A2aAgentExecutorConfig,
)
from google.adk.agents import Agent, SequentialAgent
from google.adk.agents.remote_a2a_agent import RemoteA2aAgent
from google.adk.artifacts import InMemoryArtifactService
from google.adk.memory.in_memory_memory_service import InMemoryMemoryService
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools.mcp_tool.mcp_toolset import MCPToolset
from google.adk.tools.mcp_tool.mcp_toolset import StreamableHTTPConnectionParams
import re
import warnings
warnings.filterwarnings('ignore', category=UserWarning)


## Configuration

Setting up Google Cloud and MCP server configuration.

In [6]:
# Set Google Cloud Configuration
os.environ['GOOGLE_GENAI_USE_VERTEXAI'] = 'FALSE'
os.environ['GOOGLE_CLOUD_PROJECT'] = 'app-ai-a5'
os.environ['GOOGLE_CLOUD_LOCATION'] = 'us-central1'

load_dotenv()
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get("a5-key")

print('Environment variables configured:')
print(f'GOOGLE_GENAI_USE_VERTEXAI: {os.environ["GOOGLE_GENAI_USE_VERTEXAI"]}')
print(f'GOOGLE_CLOUD_PROJECT: {os.environ["GOOGLE_CLOUD_PROJECT"]}')
print(f'GOOGLE_CLOUD_LOCATION: {os.environ["GOOGLE_CLOUD_LOCATION"]}')
print("GOOGLE_API_KEY loaded =", os.environ["GOOGLE_API_KEY"] is not None)

Environment variables configured:
GOOGLE_GENAI_USE_VERTEXAI: FALSE
GOOGLE_CLOUD_PROJECT: app-ai-a5
GOOGLE_CLOUD_LOCATION: us-central1
GOOGLE_API_KEY loaded = True


In [7]:
if 'google.colab' in sys.modules:
    from google.colab import auth

    auth.authenticate_user(project_id=os.environ['GOOGLE_CLOUD_PROJECT'])

In [ ]:
MCP_SERVER_URL = userdata.get('MCP_SERVER_URL')
print(MCP_SERVER_URL)

## A2A Communication Logger

Custom logger to track agent-to-agent communication flow.

In [8]:
# Configure basic logging
logging.basicConfig(
    level=logging.ERROR,
    format='%(asctime)s - %(levelname)s - %(name)s - %(message)s',
    force=True
)

# Add A2A Communication Logger
print("Setting up A2A Communication Logger...")

class A2ALogger:
    """Tracks agent-to-agent communication"""

    def __init__(self):
        self.request_count = 0

    def log_query(self, query: str):
        """Log user query"""
        self.request_count += 1
        print("\n" + "="*70)
        print(f"USER QUERY")
        print("="*70)
        print(f"Query: {query}")
        print("="*70 + "\n")

    def log_response(self, response: str):
        """Log final response"""
        print("\n" + "="*70)
        print(f"FINAL RESPONSE")
        print("="*70)
        print(response)
        print("="*70 + "\n")

# Initialize logger
a2a_logger = A2ALogger()
print("A2A Logger Ready!")

Setting up A2A Communication Logger...
A2A Logger Ready!


## Agent Initialization

Defining the three specialized agents that form the multi-agent customer service system:


1.   Customer Data Agent
2.   Support Agent
3.   Router (Orchestrator) Agent

Each agent has specific responsibilities and logging capabilities to track A2A communication.

### Add Customer Data Agent

Interfaces directly with the MCP server to perform all customer data operations including retrieval, updates, and ticket management. This agent announces all MCP tool calls with `[CUSTOMER_DATA_AGENT]:` prefix for visibility into data operations.

In [10]:
customer_data_agent = Agent(
    model="gemini-2.0-flash",
    name="customer_data_agent",
    tools=[ MCPToolset(
        connection_params=StreamableHTTPConnectionParams(
            url=MCP_SERVER_URL
            )
        )
    ],
    instruction="""
    You are the Customer Data Agent.

    IMPORTANT LOGGING: Begin EVERY response with:
    [CUSTOMER_DATA_AGENT]: <brief description of what you're doing>

    Your role is to interface with the MCP server and perform EXACT data operations.

    REQUIRED RULES:
    - ALWAYS announce your action first: [CUSTOMER_DATA_AGENT]: Calling get_customer for ID X
    - ALWAYS call an MCP tool for any operation involving customer data.
    - NEVER invent fields, values, or IDs.
    - ALWAYS return valid JSON after your announcement.
    - If a customer does not exist, return:
      {"error": "Customer ID not found", "customer_id": <id>}
    - For successful lookups, return:
      {"customer": { ...mcp result... }}

    AVAILABLE OPERATIONS:
    - get_customer(customer_id)
    - list_customers(status, limit)
    - update_customer(customer_id, data)
    - create_ticket(customer_id, issue, priority)
    - get_customer_history(customer_id)

    ADDITIONAL RULES:
    - When updating a customer, preserve fields not being modified.
    - When creating a ticket, always include created_at returned by MCP.
    - When listing customers, return array of customer objects.

    ALWAYS start with [CUSTOMER_DATA_AGENT]: then provide JSON.
    """
)
print("Customer Data Agent created with logging!")

Customer Data Agent created with logging!


In [11]:
customer_data_agent_card = AgentCard(
    name='Customer Data Agent',
    url='http://127.0.0.1:10020',
    description='Fetches and updates customer data using MCP tools',
    version='1.0',
    capabilities=AgentCapabilities(streaming=False),
    default_input_modes=['text/plain'],
    default_output_modes=['application/json'],
    preferred_transport=TransportProtocol.jsonrpc,
    skills=[
        AgentSkill(
            id='customer_data_access',
            name='Customer Data Access',
            description='Retrieve and update customer records',
            tags=['customer', 'data', 'database', 'lookup'],
            examples=[
                'Get customer 42',
                'Update customer 5 email',
                'Show all customers',
            ],
        )
    ],
)


In [12]:
remote_customer_data_agent = RemoteA2aAgent(
    name='customer_data_agent',
    description='Remote wrapper for Customer Data Agent',
    agent_card=f'http://127.0.0.1:10020{AGENT_CARD_WELL_KNOWN_PATH}',  # ← FIXED
)

/tmp/ipython-input-2030816957.py:1: UserWarning: [EXPERIMENTAL] RemoteA2aAgent: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  remote_customer_data_agent = RemoteA2aAgent(


### Add Support Agent

Handles customer support queries, troubleshooting, and escalations. Routes pure data retrieval requests to the Customer Data Agent, and requests customer context when needed to provide personalized support. Logs all actions with `[SUPPORT_AGENT]:` prefix.

In [13]:
support_agent = Agent(
    model="gemini-2.0-flash-exp",
    name="support_agent",
    instruction="""
    You are the Support Agent.

    IMPORTANT LOGGING: Begin EVERY response with:
    [SUPPORT_AGENT]: <brief description of what you're doing>

    CRITICAL RULE: If the user asks for customer INFORMATION/DATA (get, retrieve, show, lookup),
    you MUST respond with:

    [SUPPORT_AGENT]: This is a data retrieval request, not a support question.
    PASS_TO_CUSTOMER_DATA_AGENT

    Only handle SUPPORT questions like:
    - "How do I reset my password?"
    - "I need help with my account"
    - "I'm having trouble logging in"

    If the query is asking for customer data/information/records, say:
    [SUPPORT_AGENT]: PASS_TO_CUSTOMER_DATA_AGENT

    If customer-specific data is required for a SUPPORT question, respond with:
    [SUPPORT_AGENT]: Requesting customer data from router
    {
      "needs_customer_data": true,
      "reason": "<why>",
      "requested_fields": ["email", "tickets", "status", ...]
    }

    When the Router provides customer data for a support question:
    - Announce: [SUPPORT_AGENT]: Processing support request with customer context
    - Use only those fields
    - Provide a natural-language support answer

    Your job: troubleshooting, escalation, and answering support questions ONLY.
    For data retrieval queries, defer to customer data agent.

    ALWAYS start responses with [SUPPORT_AGENT]:
    """,
)
print("Support Agent updated with better routing logic!")

Support Agent updated with better routing logic!


In [14]:
support_agent_card = AgentCard(
    name='Support Agent',
    url='http://127.0.0.1:10021',   # FIXED
    description='Handles general support questions and escalates to customer data agent when needed',
    version='1.0',
    capabilities=AgentCapabilities(streaming=True),
    default_input_modes=['text/plain'],
    default_output_modes=['text/plain'],
    preferred_transport=TransportProtocol.jsonrpc,
    skills=[
        AgentSkill(
            id='customer_support',
            name='Customer Support',
            description='Provides general support responses',
            tags=['support', 'help', 'troubleshooting'],
            examples=[
                'How do I reset my password?',
                'What is your refund policy?',
                'I need help with my account.',
            ],
        )
    ],
)


In [15]:
remote_support_agent = RemoteA2aAgent(
    name='support_agent',
    description='Remote wrapper for Support Agent',
    agent_card=f'http://127.0.0.1:10021{AGENT_CARD_WELL_KNOWN_PATH}',  # FIXED
)

/tmp/ipython-input-332038127.py:1: UserWarning: [EXPERIMENTAL] RemoteA2aAgent: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  remote_support_agent = RemoteA2aAgent(


### Add Router (Orchestrator) Agent

Orchestrates the multi-agent system by coordinating between the Customer Data Agent and Support Agent. Built as a SequentialAgent that processes queries through specialized sub-agents to provide intelligent routing and response synthesis.

In [16]:
router_agent = SequentialAgent(
    name="router_agent_host",
    sub_agents=[customer_data_agent, support_agent]
)
print("Router Agent created!")

Router Agent created!


In [17]:
router_agent_card = AgentCard(
    name="router_agent_host",
    url="http://localhost:10040",
    description="orchestrates customer data agent and support agent, routes to support and customer-data agents, and composes the final answer.",
    version="1.0",
    capabilities=AgentCapabilities(streaming=True),
    default_input_modes=["text/plain"],
    default_output_modes=["application/json"],
    preferred_transport=TransportProtocol.jsonrpc,
    skills=[
        AgentSkill(
            id="router_orchestration",
            name="Router / Orchestrator",
            description="Routes user queries to sub-agents using A2A calls and coordinates outputs.",
            tags=["router", "orchestrator", "workflow", "a2a"],
            examples=[
                "Help me access my account",
                "Look up customer 12",
                "Update the customer's email",
                "The customer cannot log in",
            ]
        )
    ]
)

## Server Infrastructure

Setting up the A2A server infrastructure to host each agent on separate ports, enabling agent-to-agent communication via HTTP/JSON-RPC protocol.

### Server Setup Functions

Functions to create and run A2A servers for each agent.

In [18]:
def create_agent_a2a_server(agent, agent_card):
    """Create an A2A server for any ADK agent.

    Args:
        agent: The ADK agent instance
        agent_card: The ADK agent card

    Returns:
        A2AStarletteApplication instance
    """
    runner = Runner(
        app_name=agent.name,
        agent=agent,
        artifact_service=InMemoryArtifactService(),
        session_service=InMemorySessionService(),
        memory_service=InMemoryMemoryService(),
    )

    config = A2aAgentExecutorConfig()
    executor = A2aAgentExecutor(runner=runner, config=config)

    request_handler = DefaultRequestHandler(
        agent_executor=executor,
        task_store=InMemoryTaskStore(),
    )

    # Create A2A application
    return A2AStarletteApplication(
        agent_card=agent_card, http_handler=request_handler
    )

In [19]:
# Apply nest_asyncio
nest_asyncio.apply()

# Store server tasks
server_tasks: list[asyncio.Task] = []


async def run_agent_server(agent, agent_card, port) -> None:
    """Run a single agent server."""
    app = create_agent_a2a_server(agent, agent_card)

    config = uvicorn.Config(
        app.build(),
        host='127.0.0.1',
        port=port,
        log_level='warning',
        loop='none',  # Important: let uvicorn use the current loop
    )

    server = uvicorn.Server(config)
    await server.serve()


async def start_all_servers() -> None:
    """Start all servers in the same event loop."""
    tasks = [
        asyncio.create_task(
            run_agent_server(customer_data_agent, customer_data_agent_card, 10020)
        ),
        asyncio.create_task(
            run_agent_server(support_agent, support_agent_card, 10021)
        ),
        asyncio.create_task(
            run_agent_server(router_agent, router_agent_card, 10040)
        ),
    ]

    await asyncio.sleep(2)

    print('All agent servers started!')
    print('   - Customer Data Agent: http://0.0.0.0:10020')
    print('   - Support Agent:       http://0.0.0.0:10021')
    print('   - Router Agent:        http://0.0.0.0:10040')

    try:
        # await asyncio.gather(*tasks)
        await asyncio.Event().wait()
    except KeyboardInterrupt:
        print('Shutting down servers...')


def run_servers_in_background() -> None:
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    loop.run_until_complete(start_all_servers())


server_thread = threading.Thread(target=run_servers_in_background, daemon=True)
server_thread.start()

time.sleep(3)


/tmp/ipython-input-2220589873.py:19: UserWarning: [EXPERIMENTAL] A2aAgentExecutorConfig: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  config = A2aAgentExecutorConfig()
/tmp/ipython-input-2220589873.py:20: UserWarning: [EXPERIMENTAL] A2aAgentExecutor: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  executor = A2aAgentExecutor(runner=runner, config=config)


All agent servers started!
   - Customer Data Agent: http://0.0.0.0:10020
   - Support Agent:       http://0.0.0.0:10021
   - Router Agent:        http://0.0.0.0:10040


In [20]:
print('Customer Data Agent Card:')
print(customer_data_agent_card)
print('Support Agent Card:')
print(support_agent_card)
print('Host Agent Card:')
print(router_agent_card)

Customer Data Agent Card:
additional_interfaces=None capabilities=AgentCapabilities(extensions=None, push_notifications=None, state_transition_history=None, streaming=False) default_input_modes=['text/plain'] default_output_modes=['application/json'] description='Fetches and updates customer data using MCP tools' documentation_url=None icon_url=None name='Customer Data Agent' preferred_transport='JSONRPC' protocol_version='0.3.0' provider=None security=None security_schemes=None signatures=None skills=[AgentSkill(description='Retrieve and update customer records', examples=['Get customer 42', 'Update customer 5 email', 'Show all customers'], id='customer_data_access', input_modes=None, name='Customer Data Access', output_modes=None, security=None, tags=['customer', 'data', 'database', 'lookup'])] supports_authenticated_extended_card=None url='http://127.0.0.1:10020' version='1.0'
Support Agent Card:
additional_interfaces=None capabilities=AgentCapabilities(extensions=None, push_notific

### A2A Client Setup

Client for sending messages to agent servers following the A2A protocol.

In [21]:
class A2ASimpleClient:
    """A2A Simple to call A2A servers."""

    def __init__(self, default_timeout: float = 240.0):
        self._agent_info_cache: dict[
            str, dict[str, Any] | None
        ] = {}  # Cache for agent metadata
        self.default_timeout = default_timeout

    async def create_task(self, agent_url: str, message: str) -> str:
        """Send a message following the official A2A SDK pattern."""
        # Configure httpx client with timeout
        timeout_config = httpx.Timeout(
            timeout=self.default_timeout,
            connect=10.0,
            read=self.default_timeout,
            write=10.0,
            pool=5.0,
        )

        async with httpx.AsyncClient(timeout=timeout_config) as httpx_client:
            # Check if we have cached agent card data
            if (
                agent_url in self._agent_info_cache
                and self._agent_info_cache[agent_url] is not None
            ):
                agent_card_data = self._agent_info_cache[agent_url]
            else:
                # Fetch the agent card
                agent_card_response = await httpx_client.get(
                    f'{agent_url}{AGENT_CARD_WELL_KNOWN_PATH}'
                )
                agent_card_data = self._agent_info_cache[agent_url] = (
                    agent_card_response.json()
                )

            # Create AgentCard from data
            agent_card = AgentCard(**agent_card_data)

            # Create A2A client with the agent card
            config = ClientConfig(
                httpx_client=httpx_client,
                supported_transports=[
                    TransportProtocol.jsonrpc,
                    TransportProtocol.http_json,
                ],
                use_client_preference=True,
            )

            factory = ClientFactory(config)
            client = factory.create(agent_card)

            # Create the message object
            message_obj = create_text_message_object(content=message)

            # Send the message and collect responses
            responses = []
            async for response in client.send_message(message_obj):
                responses.append(response)

            # The response is a tuple - get the first element (Task object)
            if (
                responses
                and isinstance(responses[0], tuple)
                and len(responses[0]) > 0
            ):
                task = responses[0][0]  # First element of the tuple

                # Extract text: task.artifacts[0].parts[0].root.text
                try:
                    return task.artifacts[0].parts[0].root.text
                except (AttributeError, IndexError):
                    return str(task)

            return 'No response received'

In [22]:
a2a_client = A2ASimpleClient()

## Testing A2A Communication

Demonstrating the A2A communication flow with comprehensive logging that tracks routing decisions, agent actions, and multi-agent coordination patterns.

### Customer Data Agent Testing

Testing direct communication with the Customer Data Agent to verify MCP tool integration and data retrieval capabilities.

In [35]:
async def test_customer_data():
    response = await a2a_client.create_task(
        'http://127.0.0.1:10020',  # correct
        'Look up customer with id 1'
    )
    print(response)

asyncio.run(test_customer_data())

2025-11-18 20:29:12,818 - ERROR - asyncio - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7a721a2d9310>


[CUSTOMER_DATA_AGENT]: Calling get_customer for ID 1
```json
{"customer": {
    "id": 1,
    "name": "John Doe",
    "email": "new-email@example.com",
    "phone": "+1-555-0101",
    "status": "active",
    "created_at": "2025-11-17 21:35:02",
    "updated_at": "2025-11-18 19:11:17"
  }
}
```


### Support Agent Testing

Testing the Support Agent's ability to handle customer service queries and demonstrate intelligent routing when customer context is needed.

In [36]:
async def test_support_agent():
    response = await a2a_client.create_task(
        'http://localhost:10021',  # correct
        'A customer says they cannot log in. What should they do?'
    )
    print("Support Agent Response:\n", response)

asyncio.run(test_support_agent())

2025-11-18 20:29:15,694 - ERROR - asyncio - Unclosed connector
connections: ['deque([(<aiohttp.client_proto.ResponseHandler object at 0x7a7219dcddf0>, 864.219179968)])']
connector: <aiohttp.connector.TCPConnector object at 0x7a721a2da1b0>


Support Agent Response:
 [SUPPORT_AGENT]: Responding to user's login issue.
Requesting customer data from router
```json
{
  "needs_customer_data": true,
  "reason": "troubleshooting login issues",
  "requested_fields": ["email", "status", "last_login", "login_attempts"]
}
```


### Router Agent Testing

Testing the Router's orchestration capabilities by coordinating between multiple agents for queries requiring both data retrieval and support assistance.

In [37]:
async def test_router_agent():
    response = await a2a_client.create_task(
        'http://localhost:10040',
        'Update customer 1 email to new-email@example.com'
    )
    print("Router Agent Response:\n", response)

asyncio.run(test_router_agent())

2025-11-18 20:29:21,255 - ERROR - asyncio - Unclosed connector
connections: ['deque([(<aiohttp.client_proto.ResponseHandler object at 0x7a7219dccdd0>, 869.827693845)])']
connector: <aiohttp.connector.TCPConnector object at 0x7a7213d58590>


Router Agent Response:
 [SUPPORT_AGENT]: This is a data retrieval request, not a support question. 
PASS_TO_CUSTOMER_DATA_AGENT


## Assignment Test Scenarios

Demonstrating the three required coordination patterns from the assignment.

In [64]:
async def run_a2a_system(query: str):
    """
    Consolidated A2A test function with full logging and intelligent routing.
    Args:
        query: User query string (may contain customer ID within the text)
    """
    # Log the initial query
    a2a_logger.log_query(query)

    # Analyze query to determine routing
    query_lower = query.lower()

    # Try to extract customer_id from query text
    customer_id = None
    if 'customer id' in query_lower or 'id' in query_lower:
        id_match = re.search(r'(?:customer\s*)?id\s*(?:is\s*)?(\d+)', query_lower)
        if id_match:
            customer_id = int(id_match.group(1))
            print(f"[ROUTER] Extracted customer_id: {customer_id} from query\n")

    # Check if query mentions customers/data
    mentions_customer_data = any(word in query_lower for word in [
        'customer', 'customers', 'ticket', 'tickets', 'email', 'phone',
        'status', 'name', 'names', 'id', 'information', 'data', 'record'
    ])

    # Check if this is a pure data retrieval query
    is_data_query = (
        any(word in query_lower for word in [
            'get', 'retrieve', 'show', 'list', 'find', 'lookup', 'update',
            'create ticket', 'what are', 'which', 'all'
        ]) and mentions_customer_data
    )

    # Check if this is a support query
    is_support_query = any(word in query_lower for word in [
        'help', 'issue', 'problem', 'trouble', 'how do i', 'cant', "can't",
        'unable', 'reset', 'refund', 'cancel', 'charged', 'need', 'billing'
    ])

    # Route based on query type
    if ('all' in query_lower and 'tickets' in query_lower and 'priority' in query_lower):
        # Multi-step coordination
        print("[ROUTER] Query type: Multi-step coordination")
        print("[ROUTER] Strategy: Decompose into sub-tasks\n")

        # Get active customers
        print("[ROUTER] Step 1: Fetching active customers")
        customers_response = await a2a_client.create_task('http://127.0.0.1:10020', "List all active customers")

        # Parse customer IDs
        import json
        customer_ids = []
        try:
            data = json.loads(re.search(r'\{.*\}', customers_response, re.DOTALL).group())
            customer_ids = [c.get('id') or c.get('customer_id') for c in data.get('customers', [])]
        except:
            pass

        print(f"[ROUTER] Step 2: Found {len(customer_ids)} customers, fetching tickets")

        # Get tickets for each customer
        high_priority_tickets = []
        for cust_id in customer_ids[:10]:  # Limit to 10
            ticket_response = await a2a_client.create_task('http://127.0.0.1:10020', f"Get customer history for customer {cust_id}")
            try:
                ticket_data = json.loads(re.search(r'\{.*\}', ticket_response, re.DOTALL).group())
                for ticket in ticket_data.get('history', []):
                    if ticket.get('priority') == 'high':
                        high_priority_tickets.append(f"Customer {cust_id}: Ticket #{ticket['id']} - {ticket['issue']} (Status: {ticket['status']})")
            except:
                pass

        print(f"[ROUTER] Step 3: Found {len(high_priority_tickets)} high-priority tickets\n")
        response = f"Found {len(high_priority_tickets)} high-priority tickets:\n" + "\n".join(high_priority_tickets) if high_priority_tickets else "No high-priority tickets found."

    elif is_data_query and not is_support_query:
        # Simple data query - route to customer data agent
        print("[ROUTER] Query type: Data retrieval")
        print("[ROUTER] Routing to: customer_data_agent")
        print(f"[ROUTER] Action: Direct call\n")

        response = await a2a_client.create_task(
            'http://127.0.0.1:10020',
            query
        )

    elif is_support_query and customer_id:
        # Support query with customer context - multi-step coordination
        print("[ROUTER] Query type: Support with customer context")
        print("[ROUTER] Strategy: Multi-agent coordination")
        print(f"[ROUTER] Step 1/2: Fetching customer {customer_id} data\n")

        # Step 1: Get customer data
        customer_data_response = await a2a_client.create_task(
            'http://127.0.0.1:10020',
            f"Get customer {customer_id}"
        )

        print(f"\n[ROUTER] Customer data received")
        print(f"[ROUTER] Step 2/2: Routing to support_agent with context\n")

        # Step 2: Pass to support agent with context
        support_query = f"{query}\n\nCustomer context: {customer_data_response}"
        response = await a2a_client.create_task(
            'http://127.0.0.1:10021',
            support_query
        )

    elif is_support_query:
        # General support query without specific customer
        print("[ROUTER] Query type: General support")
        print("[ROUTER] Routing to: support_agent")
        print(f"[ROUTER] Action: Direct call\n")

        response = await a2a_client.create_task(
            'http://127.0.0.1:10021',
            query
        )

    else:
        # Default to router agent
        print("[ROUTER] Query type: Complex/Ambiguous")
        print("[ROUTER] Routing to: router_agent_host")
        print(f"[ROUTER] Action: Let router orchestrate\n")

        response = await a2a_client.create_task(
            'http://localhost:10040',
            query
        )

    # Log final response
    a2a_logger.log_response(response)

    return response

print("Consolidated A2A test function ready!")

2025-11-18 22:05:01,765 - ERROR - asyncio - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7a7213de7c80>


Consolidated A2A test function ready!


### Scenario 1: Task Allocation

**Query**: "I need help with my account, customer ID 1"

**Expected A2A Flow**:
1. Router receives query
2. Router → Customer Data Agent: Get customer info for ID 1
3. Customer Data Agent fetches data via MCP
4. Customer Data Agent → Router: Returns customer data
5. Router → Support Agent: Handle support with customer context
6. Support Agent generates response
7. Router returns final response

In [65]:
await run_a2a_system("Get customer information for ID 1")


USER QUERY #25
Query: Get customer information for ID 1

[ROUTER] Extracted customer_id: 1 from query

[ROUTER] Query type: Data retrieval
[ROUTER] Routing to: customer_data_agent
[ROUTER] Action: Direct call


FINAL RESPONSE
[CUSTOMER_DATA_AGENT]: Calling get_customer for ID 1
```json
{"customer": {"id": 1, "name": "John Doe", "email": "new-email@example.com", "phone": "+1-555-0101", "status": "active", "created_at": "2025-11-17 21:35:02", "updated_at": "2025-11-18 20:29:19"}}
```



'[CUSTOMER_DATA_AGENT]: Calling get_customer for ID 1\n```json\n{"customer": {"id": 1, "name": "John Doe", "email": "new-email@example.com", "phone": "+1-555-0101", "status": "active", "created_at": "2025-11-17 21:35:02", "updated_at": "2025-11-18 20:29:19"}}\n```'

### Scenario 2: Negotiation/Escalation

**Query**: "I want to cancel my subscription but I'm having billing issues"

**Expected A2A Flow**:
1. Router detects multiple intents (cancellation + billing)
2. Router → Support Agent: Initial assessment
3. Support Agent → Router: Requests customer context
4. Router → Customer Data Agent: Get customer/billing info
5. Router coordinates response between agents
6. Final response addresses both concerns

In [66]:
await run_a2a_system("I want to cancel my subscription but I'm having billing issues")

2025-11-18 22:05:15,581 - ERROR - asyncio - Unclosed connector
connections: ['deque([(<aiohttp.client_proto.ResponseHandler object at 0x7a7210bd5a90>, 6615.285676657)])']
connector: <aiohttp.connector.TCPConnector object at 0x7a721a1f3bf0>



USER QUERY #26
Query: I want to cancel my subscription but I'm having billing issues

[ROUTER] Query type: General support
[ROUTER] Routing to: support_agent
[ROUTER] Action: Direct call


FINAL RESPONSE
[SUPPORT_AGENT]: Requesting customer data from router
```json
{
  "needs_customer_data": true,
  "reason": "Need to check subscription status and billing details to assist with cancellation and billing issues.",
  "requested_fields": ["subscription_status", "billing_information"]
}
```



'[SUPPORT_AGENT]: Requesting customer data from router\n```json\n{\n  "needs_customer_data": true,\n  "reason": "Need to check subscription status and billing details to assist with cancellation and billing issues.",\n  "requested_fields": ["subscription_status", "billing_information"]\n}\n```'

### Scenario 3: Multi-Step Coordination

**Query**: "Show me all active customers who have open tickets"

**Expected A2A Flow**:
1. Router decomposes query into sub-tasks
2. Router → Customer Data Agent: Get all active customers
3. Customer Data Agent → Router: Returns customer list
4. Router → Customer Data Agent: Get ticket history for each customer
5. Router filters for open tickets
6. Router synthesizes final report

In [67]:
await run_a2a_system("What's the status of all high-priority tickets for active customers?")

2025-11-18 22:05:20,235 - ERROR - asyncio - Unclosed connector
connections: ['deque([(<aiohttp.client_proto.ResponseHandler object at 0x7a7210bd5af0>, 6625.074673204)])']
connector: <aiohttp.connector.TCPConnector object at 0x7a7219e94560>



USER QUERY #27
Query: What's the status of all high-priority tickets for active customers?

[ROUTER] Query type: Multi-step coordination
[ROUTER] Strategy: Decompose into sub-tasks

[ROUTER] Step 1: Fetching active customers
[ROUTER] Step 2: Found 0 customers, fetching tickets
[ROUTER] Step 3: Found 0 high-priority tickets


FINAL RESPONSE
No high-priority tickets found.



'No high-priority tickets found.'

In [68]:
# Debug: See what the customer data agent returns
customers_response = await a2a_client.create_task(
    'http://127.0.0.1:10020',
    "List all active customers"
)
print("Raw response:")
print(customers_response)

2025-11-18 22:05:58,009 - ERROR - asyncio - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7a721a1f23f0>


Raw response:
[CUSTOMER_DATA_AGENT]: Calling list_customers with status active.
```json
{"customers": []}
```
